#  Deploying Predictive Systems

Este notebook es una prueba de concepto para verificar el funcionamiento del modelo predictivo generado durante el análisis exploratorio previo. Aquí no se realiza un despliegue en producción ni se integra con Kafka o sistemas de streaming: el objetivo es simplemente cargar el modelo ya entrenado y comprobar que genera predicciones razonables sobre el dataset.

---

##  Configuración del entorno Spark

Creamos una nueva sesión de Spark apuntando al clúster especificado. Es importante cerrar cualquier contexto activo anterior para evitar conflictos. Este entorno nos permitirá procesar el DataFrame y generar predicciones con el modelo entrenado.

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext

# Cerrar SparkContext anterior si está activo
if SparkContext._active_spark_context:
    SparkContext._active_spark_context.stop()

# SparkSession apuntando al cluster
spark = (
    SparkSession.builder.appName("DeployMyModel")
    .master("spark://agile:7077")  # Asegúrate de que 'agile' es el nombre correcto del master en tu Docker Compose
    .config("spark.driver.bindAddress", "0.0.0.0")
    .getOrCreate()
)

sc = spark.sparkContext
sc.setLogLevel("ERROR")

###  Verificación de la conexión con Spark

Comprobamos que Spark está correctamente conectado al clúster y validamos la información básica de la sesión.

In [ ]:
print(spark.version)
print(spark.sparkContext.master)
print("Aplicación:", spark.sparkContext.appName)
print(spark.sparkContext._jsc.sc().getExecutorMemoryStatus().keys())

##  Carga del dataset de prueba

En esta prueba usaremos un subconjunto del dataset original utilizado en el análisis exploratorio. Este paso intenta copiar el fichero a la ubicación deseada, aunque puede omitirse si ya se dispone del archivo en `./data/`.

In [ ]:
!cp "/home/jovyan/.cache/kagglehub/datasets/romanniki/food-delivery-cost-and-profitability/versions/1/food_orders_new_delhi (1).csv" ./data/food_orders.csv

###  Lectura de datos

Leemos el archivo `food_orders.csv` como un DataFrame de Spark, infiriendo automáticamente el esquema. Mostramos las primeras filas para validar su contenido.

In [ ]:
input_path = "./data/food_orders.csv"
df = spark.read.csv(input_path, header=True, inferSchema=True)
df.show(5)

##  Enriquecimiento del dataset

Añadimos columnas calculadas a partir de la fecha y otros campos para generar variables que el modelo necesita: día de la semana, hora del pedido, si fue en fin de semana, si fue en hora punta, si tenía descuento, etc.

In [ ]:
from pyspark.sql.functions import dayofweek, hour, when, col

# Añadir columnas calculadas en Spark
df = df.withColumn("day_of_week", dayofweek("Order Date and Time"))  # 1=Sunday, ..., 7=Saturday
df = df.withColumn("hour_of_day", hour("Order Date and Time"))

# Es fin de semana (sábado=7 o domingo=1)
df = df.withColumn("es_fin_de_semana", when(col("day_of_week").isin([1, 7]), 1).otherwise(0))

# Es hora punta (13-15 o 20-22)
df = df.withColumn(
    "es_hora_punta",
    when((col("hour_of_day").between(13, 15)) | (col("hour_of_day").between(20, 22)), 1).otherwise(0)
)

In [ ]:
df.show(5)

In [ ]:
df = df.withColumnRenamed("Order ID", "order_id") \
       .withColumnRenamed("Customer ID", "customer_id") \
       .withColumnRenamed("Restaurant ID", "restaurant_id") \
       .withColumnRenamed("Order Date and Time", "order_date_and_time") \
       .withColumnRenamed("Delivery Date and Time", "delivery_date_and_time") \
       .withColumnRenamed("Order Value", "order_value") \
       .withColumnRenamed("Delivery Fee", "delivery_fee") \
       .withColumnRenamed("Payment Method", "payment_method") \
       .withColumnRenamed("Discounts and Offers", "discounts_and_offers") \
       .withColumnRenamed("Commission Fee", "commission_fee") \
       .withColumnRenamed("Payment Processing Fee", "payment_processing_fee") \
       .withColumnRenamed("Refunds/Chargebacks", "refunds/chargebacks")

In [ ]:
from pyspark.sql.functions import dayofweek, hour, when, col

# Día de la semana y hora
df = df.withColumn("day_of_week", dayofweek("order_date_and_time"))
df = df.withColumn("hour_of_day", hour("order_date_and_time"))

# Fin de semana
df = df.withColumn("es_fin_de_semana", when(col("day_of_week").isin([1, 7]), 1).otherwise(0))

# Hora punta
df = df.withColumn("es_hora_punta", when(
    (col("hour_of_day").between(13, 15)) | (col("hour_of_day").between(20, 22)), 1
).otherwise(0))

# Tiene descuento
df = df.withColumn("has_discount", when(
    col("discounts_and_offers").isNotNull() & (col("discounts_and_offers") != "None"), 1
).otherwise(0))

# Valor del descuento
df = df.withColumn("discount_value", col("order_value") * col("has_discount").cast("int") * 0.1)

df = df.withColumn("refunded", (col("refunds/chargebacks") > 0).cast("boolean"))

##  Carga y aplicación del modelo

Cargamos el pipeline de Spark MLlib previamente entrenado (incluye transformadores + modelo) y lo aplicamos al DataFrame enriquecido para obtener predicciones sobre la duración estimada del pedido.

In [ ]:
from pyspark.ml import PipelineModel

modelo = PipelineModel.load("./models/pipeline_model.bin")
predicciones = modelo.transform(df)

predicciones.select("prediction").show(10)

###  Predicciones generadas

Mostramos algunas de las predicciones generadas por el modelo. En este caso, la variable objetivo es la duración estimada del pedido (en minutos, por ejemplo), y el modelo devuelve un valor numérico continuo para cada fila del dataset.

##  Conclusión

Este notebook permite validar que el modelo entrenado previamente funciona correctamente cuando se aplica a nuevos datos. Para escenarios reales, sería necesario integrarlo con pipelines de streaming, almacenamiento persistente y visualización de resultados.